In [25]:
# ===============================
# 1. 필요한 라이브러리 import
# ===============================


import os
# LangChain v1 agent 생성 함수
from langchain.agents import create_agent
# OpenAI chat 모델 래퍼
from langchain_openai import ChatOpenAI

In [26]:
# 외부 정보 수집용 tool들
from langchain_community.tools import (
    WikipediaQueryRun, # 위키피디아 검색/요약 tool
    DuckDuckGoSearchRun, # 웹 최신 검색 tool
    RequestsGetTool, # 특정 URL에 HTTP GET 요청을 보내는 tool
)

# 위키피디아 API Wrapper
from langchain_community.utilities import WikipediaAPIWrapper

In [43]:
# ===============================
# 2. OpenAI LLM 설정
# ===============================

# OpenAI의 Chat 모델을 LangChain에서 사용하기 위한 객체 생성
# - model="gpt-4o-mini": 비교적 저렴하고 tool 호출을 잘 지원하는 chat 모델
# - temperature=0: 출력의 랜덤성을 줄이고, 재현성 있는 응답을 얻기 위함
# 이 API는 나중에 본인걸로 바꾸어주세요..
# (권장 사항)
# API 키는 코드에 직접 쓰지 말고,
# 터미널에서 export OPENAI_API_KEY=... 한 뒤 주피터를 실행하는 것이 안전함.
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment"

model = ChatOpenAI(model="gpt-4o-mini", temperature=0) # Or "gpt-3.5-turbo-instruct", temperature=0) # Or gtp-4o-mini

In [33]:
# ===============================
# 3. HTTP 요청 tool 설정
# ===============================

# RequestsGetTool은 임의의 URL로 HTTP 요청을 보낼 수 있기 때문에 
# 보안상 명시적으로 "위험함을 인지했다"는 플래그를 켜야 함. (allow_dangerous_requests)

from langchain_community.tools import RequestsGetTool
from langchain_community.utilities import RequestsWrapper

api_tool = RequestsGetTool(
    requests_wrapper=RequestsWrapper(),
    allow_dangerous_requests=True,
)


In [34]:
# ===============================
# 4. Agent가 사용할 tool 목록 정의
# ===============================

# 이 리스트에 들어간 tool들 중에서,
# agent(LLM)가 "이 상황에 어떤 tool을 쓰는 게 좋을까?"를 스스로 판단해서 사용함.


tools = [
    WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
    DuckDuckGoSearchRun(),
    api_tool,
]

In [35]:
# ===============================
# 5. Agent 생성
# ===============================

# create_agent는
# - LLM(model)
# - 사용할 tool 목록(tools)
# - system prompt(에이전트의 기본 행동 지침)
# 을 묶어서 하나의 "에이전트"로 만들어준다.



# Agent (LangChain v1 방식)
agent = create_agent(
    model,
    tools=tools,
    system_prompt="You are a helpful assistant. Use tools when useful."
)

In [44]:
# Run
# agent에게 실제로 질문을 던지고, tool을 써서 답을 만들어오게 하는 실행 단계

# 1) agent.invoke(...)는 "agent를 한 번 실행"하는 호출.
#    - 입력으로 대화 메시지 목록(messages)을 넘긴다.
#    - agent는 메시지를 읽고, 필요하면 tools(Wikipedia/DDG/RequestsGetTool)를 선택해 호출한 뒤,
#      최종 답변을 만들어 result로 돌려준다.
result = agent.invoke({
    "messages": [
        {
            # 2) role="user"는 "사용자 발화"라는 뜻.
            #    agent는 이 메시지를 보고 무엇을 해야할지 판단한다.
            "role": "user",

            # 3) 실제 사용자 요청 텍스트.
            #    여기서는 (a) 위키 요약 + (b) 웹 최신 뉴스 검색 + (c) NASA API 홈페이지 fetch를 요구한다.
            "content": "What's the latest news about NASA on Wikipedia and the web? Also, fetch the NASA API homepage."
        }
    ]
})

# 4) result는 보통 dict 형태로 반환되며,
#    - messages: 실행 과정에서 쌓인 메시지들(사용자/도구 호출/도구 결과/최종 답변 등)
#    - 기타 메타데이터(버전에 따라 다름)
#    를 포함한다.
print(result)

# (참고) 대부분은 아래처럼 마지막 assistant 메시지의 텍스트만 뽑아 보는 게 더 편하다:
# print(result["messages"][-1].content)


/Users/sookim/miniconda3/envs/mcp/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/sookim/miniconda3/envs/mcp/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


{'messages': [HumanMessage(content="What's the latest news about NASA on Wikipedia and the web? Also, fetch the NASA API homepage.", additional_kwargs={}, response_metadata={}, id='7171a2cd-4c15-4510-bef3-d8f6133a6e60'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 226, 'total_tokens': 295, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f4ae844694', 'id': 'chatcmpl-D6s0jdD7bXty3OAmhL3dCPKId4JiP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3bcf-2477-7280-b24d-e7e3cd40fd28-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'NASA'}, 'id': 'call_Dxv1q6axyaOyYXH7Jwk5kfP7', 'type': 'tool_call'}, {'na

In [45]:
print(result)

{'messages': [HumanMessage(content="What's the latest news about NASA on Wikipedia and the web? Also, fetch the NASA API homepage.", additional_kwargs={}, response_metadata={}, id='7171a2cd-4c15-4510-bef3-d8f6133a6e60'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 226, 'total_tokens': 295, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f4ae844694', 'id': 'chatcmpl-D6s0jdD7bXty3OAmhL3dCPKId4JiP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3bcf-2477-7280-b24d-e7e3cd40fd28-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'NASA'}, 'id': 'call_Dxv1q6axyaOyYXH7Jwk5kfP7', 'type': 'tool_call'}, {'na

In [41]:
# 실행 후 메시지들을 훑어서 tool 메시지/assistant 메시지 등을 확인
for m in result["messages"]:
    print(m.type, ":", (m.content[:120] + "...") if isinstance(m.content, str) and len(m.content) > 120 else m.content)


human : What's the latest news about NASA on Wikipedia and the web? Also, fetch the NASA API homepage.
ai : 
tool : Page: NASA
Summary: The National Aeronautics and Space Administration (NASA ) is an independent agency of the U.S. feder...
tool : Jun 17, 2025 · Stay up-to-date with the latest content from NASA as we explore the universe and discover more about our ...
tool : <!doctype html>
<base href="#">
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>NASA Open APIs</title>
  <link...
ai : ### Latest Information about NASA

#### Wikipedia Summary
The National Aeronautics and Space Administration (NASA) is an...


In [42]:
print(result["messages"][-1].content)

### Latest Information about NASA

#### Wikipedia Summary
The National Aeronautics and Space Administration (NASA) is an independent agency of the U.S. federal government responsible for the United States' civil space program and for research in aeronautics and space exploration. Established in 1958, NASA has led most of America's space exploration programs, including Project Mercury, Project Gemini, the Apollo program, Skylab, and the Space Shuttle. It operates ten field centers across the U.S. and maintains major ground and communications infrastructure, including the Deep Space Network.

NASA's science division focuses on understanding Earth, heliophysics, exploring the Solar System with robotic spacecraft, and researching astrophysics topics through telescopes like the James Webb Space Telescope. The agency supports the International Space Station and oversees the development of the Orion spacecraft and the Space Launch System for the lunar Artemis program. For FY2022, NASA had a b